In [ ]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import glob
import os
from skimage import io
from keras.preprocessing.image import ImageDataGenerator
!ls 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train'

NORMAL	PNEUMONIA


In [ ]:
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
"""
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
"""

"\ntry:\n  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection\n  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])\nexcept ValueError:\n  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')\n\ntf.config.experimental_connect_to_cluster(tpu)\ntf.tpu.experimental.initialize_tpu_system(tpu)\ntpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)\n"

In [ ]:
train_normal = np.array(glob.glob('drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/*.jpeg'))
train_pneumonia = np.array(glob.glob('drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/*.jpeg'))

#train_pneumonia = np.random.choice(train_pneumonia, size = 1859)

print(train_normal)
print(train_pneumonia)

print(train_normal.shape)
print(train_pneumonia.shape)
print(train_normal.shape[0] + train_pneumonia.shape[0])

['drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/NORMAL2-IM-0820-0001.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/NORMAL2-IM-0821-0001.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/NORMAL2-IM-0822-0001.jpeg'
 ...
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/IM-0591-0001.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/IM-0592-0001.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/IM-0596-0001.jpeg']
['drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person73_bacteria_356.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person73_bacteria_357.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person73_bacteria_358.jpeg'
 ...
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person1109_bacteria_3050.jpeg'
 'drive/MyDrive/Winter P

In [ ]:
df_train = np.concatenate((train_normal, train_pneumonia), axis=None)
print(df_train)

['drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/NORMAL2-IM-0820-0001.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/NORMAL2-IM-0821-0001.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/NORMAL/NORMAL2-IM-0822-0001.jpeg'
 ...
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person1109_bacteria_3050.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person110_virus_205.jpeg'
 'drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/train/PNEUMONIA/person110_virus_206.jpeg']


In [ ]:
df_train = pd.DataFrame(df_train, columns=['label'])
df_train.head()

,label
0,drive/MyDrive/Winter Program 2021/Lung/Data/ch...
1,drive/MyDrive/Winter Program 2021/Lung/Data/ch...
2,drive/MyDrive/Winter Program 2021/Lung/Data/ch...
3,drive/MyDrive/Winter Program 2021/Lung/Data/ch...
4,drive/MyDrive/Winter Program 2021/Lung/Data/ch...


In [ ]:
def DefineID(inpIteration):
  if("NORMAL" in inpIteration):
    return "NORMAL"
  else:
    return "PNEUMONIC"

In [ ]:
#df_train.loc['NORMAL' in df_train['label'], 'id'] = 0
#df_train.loc['PNEUMONIA' in df_train['label'], 'id'] = 1
#df_train.head()
#df_train['id'] = np.where('NORMAL' in df_train['label'], 1, 0)
df_train["id"] = df_train["label"].apply(DefineID)
df_train.head()

,label,id
0,drive/MyDrive/Winter Program 2021/Lung/Data/ch...,NORMAL
1,drive/MyDrive/Winter Program 2021/Lung/Data/ch...,NORMAL
2,drive/MyDrive/Winter Program 2021/Lung/Data/ch...,NORMAL
3,drive/MyDrive/Winter Program 2021/Lung/Data/ch...,NORMAL
4,drive/MyDrive/Winter Program 2021/Lung/Data/ch...,NORMAL


In [ ]:
#print(df_train.loc[5000, 'id'])
#print(type(df_train.loc[5000, 'id']))

In [ ]:
print(df_train.shape)

(5216, 2)


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_gen = train_datagen.flow_from_dataframe(dataframe=df_train, x_col='label', y_col='id', target_size=(200, 200), class_mode="binary", shuffle=True)
#steps_per_epoch= 326, batch_size= 16

Found 5216 validated image filenames belonging to 2 classes.


In [ ]:
vgg = tf.keras.applications.VGG16(include_top=False, input_shape=(200, 200, 3))

In [ ]:
for i in vgg.layers:
  i.trainable = True

In [ ]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

In [ ]:
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(4096, activation = 'relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 6, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              1052672   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4097      
Total params: 15,902,785
Trainable params: 15,902,785
Non-trainable params: 0
____________________________________________

In [ ]:
checkpoint_path_colab = "training_cp/cp-{epoch:03d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_colab, save_weights_only=True)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
train_gen.reset()

In [ ]:
model.fit(train_gen, epochs = 50, callbacks=[cp_callback], batch_size=32)

Epoch 1/50
163/163 [==============================] - 4690s 29s/step - loss: 0.3812 - accuracy: 0.8185
Epoch 2/50
163/163 [==============================] - 99s 604ms/step - loss: 0.2236 - accuracy: 0.9057
Epoch 3/50
163/163 [==============================] - 98s 602ms/step - loss: 0.1729 - accuracy: 0.9300
Epoch 4/50
163/163 [==============================] - 99s 604ms/step - loss: 0.1340 - accuracy: 0.9423
Epoch 5/50
163/163 [==============================] - 99s 604ms/step - loss: 0.1085 - accuracy: 0.9571
Epoch 6/50
163/163 [==============================] - 98s 602ms/step - loss: 0.0989 - accuracy: 0.9610
Epoch 7/50
163/163 [==============================] - 99s 604ms/step - loss: 0.0958 - accuracy: 0.9645
Epoch 8/50
163/163 [==============================] - 99s 604ms/step - loss: 0.0728 - accuracy: 0.9734
Epoch 9/50
163/163 [==============================] - 98s 601ms/step - loss: 0.0769 - accuracy: 0.9706
Epoch 10/50
163/163 [==============================] - 98s 603ms/step - l

KeyboardInterrupt: ignored

In [ ]:
test_normal = np.array(glob.glob('drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/test/NORMAL/*.jpeg'))
test_pneumonia = np.array(glob.glob('drive/MyDrive/Winter Program 2021/Lung/Data/chest_xray/test/PNEUMONIA/*.jpeg'))
df_test = np.concatenate((test_normal, test_pneumonia), axis=None)
df_test = pd.DataFrame(df_test, columns=['label'])
print(df_test)
df_test["id"] = df_test["label"].apply(DefineID)
test_gen = test_datagen.flow_from_dataframe(dataframe=df_test, x_col='label', y_col='id', target_size=(200, 200), class_mode="binary", shuffle=False)

                                                 label
0    drive/MyDrive/Winter Program 2021/Lung/Data/ch...
1    drive/MyDrive/Winter Program 2021/Lung/Data/ch...
2    drive/MyDrive/Winter Program 2021/Lung/Data/ch...
3    drive/MyDrive/Winter Program 2021/Lung/Data/ch...
4    drive/MyDrive/Winter Program 2021/Lung/Data/ch...
..                                                 ...
619  drive/MyDrive/Winter Program 2021/Lung/Data/ch...
620  drive/MyDrive/Winter Program 2021/Lung/Data/ch...
621  drive/MyDrive/Winter Program 2021/Lung/Data/ch...
622  drive/MyDrive/Winter Program 2021/Lung/Data/ch...
623  drive/MyDrive/Winter Program 2021/Lung/Data/ch...

[624 rows x 1 columns]
Found 624 validated image filenames belonging to 2 classes.


In [ ]:
model.load_weights('/content/training_cp/cp-014.ckpt')

In [ ]:
model.evaluate(test_gen, verbose=1)

20/20 [==============================] - 6s 269ms/step - loss: 0.2153 - accuracy: 0.9327


[0.2152969390153885, 0.932692289352417]

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = df_test['id'].to_numpy()
print(y_true)
test_gen.reset()
y_hat = np.rint(model.predict(test_gen).transpose()[0])
print(y_hat)


['NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL'
 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORMAL' 'NORM

In [ ]:
y_true_num = []
for i in y_true:
  if i == "NORMAL":
    y_true_num.append(0)
  else:
    y_true_num.append(1)

confusion=confusion_matrix(y_true_num, y_hat)
print(confusion)

[[195  39]
 [  3 387]]


In [ ]:
vgg_2 = tf.keras.applications.VGG16(include_top=False, input_shape=(200, 200, 3))
model_2 = tf.keras.Sequential()
model_2.add(vgg)
model_2.add(tf.keras.layers.)
model_2.add(tf.keras.layers.Dense(256, activation = 'relu'))
model_2.add(tf.keras.layers.Dropout(0.3))
model_2.add(tf.keras.layers.Dense(4096, activation = 'relu'))
model_2.add(tf.keras.layers.Dense(1, activation='sigmoid'))